In [1]:
import pandas as pd
import requests
import time

In [12]:

# Schritt 1: Daten einlesen
country_df = pd.read_csv(r'C:\Users\Admin\Desktop\07 Python für Datenanalysten\Projekt_UFO\Nadja\merged_data_ufo_sightings.csv')

# Funktion zur Bestimmung des Landes mit Geonames API
def get_country_from_geonames(city):
    username = "city_country"  # Ersetzen Sie dies durch Ihren tatsächlichen Geonames-Benutzernamen
    url = f"http://api.geonames.org/searchJSON?q={city}&maxRows=1&username={username}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Überprüfen Sie, ob die Anfrage erfolgreich war
        data = response.json()
        if 'geonames' in data and len(data['geonames']) > 0:
            return data['geonames'][0]['countryName']
        else:
            print(f"Keine Ergebnisse für Stadt '{city}' gefunden.")
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP-Fehler bei der Anfrage für Stadt '{city}': {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Verbindungsfehler bei der Anfrage für Stadt '{city}': {conn_err}")
    except requests.exceptions.Timeout:
        print(f"Timeout-Fehler bei der Anfrage für Stadt '{city}'.")
    except requests.exceptions.RequestException as req_err:
        print(f"Ein Fehler ist bei der Anfrage für Stadt '{city}' aufgetreten: {req_err}")
    return None

# Funktion zur Bestimmung des Landes mit REST Countries API
def get_country_from_rest_countries(city):
    url = f"https://restcountries.com/v3.1/capital/{city}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Überprüfen Sie, ob die Anfrage erfolgreich war
        data = response.json()
        if data:
            return data[0]['name']['common']  # Gibt den Namen des Landes zurück
    except requests.exceptions.RequestException as e:
        print(f"Fehler bei der Anfrage für Stadt '{city}': {e}")
    return None

# Schritt 2: Vervollständigung der fehlenden Werte
for index, row in country_df[country_df['country'].isnull()].iterrows():
    city = row['city']
    country = get_country_from_geonames(city)
    if not country:
        country = get_country_from_rest_countries(city)
    if country:
        country_df.at[index, 'country'] = country
    time.sleep(1)  # Warte 1 Sekunde zwischen den Anfragen, um Ratenbegrenzungen zu vermeiden

# Schritt 3: Daten speichern
country_df.to_csv(r'C:\Users\Admin\Desktop\07 Python für Datenanalysten\Projekt_UFO\Nadja\countries.csv', index=False)


HTTP-Fehler bei der Anfrage für Stadt 'lackland afb': 401 Client Error: 401 for url: http://api.geonames.org/searchJSON?q=lackland%20afb&maxRows=1&username=city_country
Fehler bei der Anfrage für Stadt 'lackland afb': 404 Client Error: Not Found for url: https://restcountries.com/v3.1/capital/lackland%20afb
HTTP-Fehler bei der Anfrage für Stadt 'lackland air force base tx': 401 Client Error: 401 for url: http://api.geonames.org/searchJSON?q=lackland%20air%20force%20base%20tx&maxRows=1&username=city_country
Fehler bei der Anfrage für Stadt 'lackland air force base tx': 404 Client Error: Not Found for url: https://restcountries.com/v3.1/capital/lackland%20air%20force%20base%20tx
HTTP-Fehler bei der Anfrage für Stadt 'bermuda nas': 401 Client Error: 401 for url: http://api.geonames.org/searchJSON?q=bermuda%20nas&maxRows=1&username=city_country
Fehler bei der Anfrage für Stadt 'bermuda nas': 404 Client Error: Not Found for url: https://restcountries.com/v3.1/capital/bermuda%20nas
HTTP-Fehl

KeyboardInterrupt: 

In [14]:
import pandas as pd
import requests
import time
from concurrent.futures import ThreadPoolExecutor

# Schritt 1: Daten einlesen
country_df = pd.read_csv(r'C:\Users\Admin\Desktop\07 Python für Datenanalysten\Projekt_UFO\Nadja\merged_data_ufo_sightings.csv')

# Funktion zur Bestimmung des Landes mit Geonames API
def get_country_from_geonames(city):
    username = 'city_country'  # Ersetzen Sie dies durch Ihren tatsächlichen Geonames-Benutzernamen
    url = f"http://api.geonames.org/searchJSON?q={city}&maxRows=1&username={username}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if 'geonames' in data and len(data['geonames']) > 0:
            return data['geonames'][0]['countryName']
        else:
            print(f"Keine Ergebnisse für Stadt '{city}' gefunden.")
    except requests.exceptions.RequestException as e:
        print(f"Fehler bei der Anfrage für Stadt '{city}': {e}")
    return None

# Funktion zur Bestimmung des Landes mit REST Countries API
def get_country_from_rest_countries(city):
    url = f"https://restcountries.com/v3.1/capital/{city}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if data:
            return data[0]['name']['common']
    except requests.exceptions.RequestException as e:
        print(f"Fehler bei der Anfrage für Stadt '{city}': {e}")
    return None

# Funktion zur Bearbeitung einer Stadt
def process_city(city):
    country = get_country_from_geonames(city)
    if not country:
        country = get_country_from_rest_countries(city)
    return country

# Schritt 2: Vervollständigung der fehlenden Werte
with ThreadPoolExecutor() as executor:
    cities = country_df[country_df['country'].isnull()]['city']
    results = list(executor.map(process_city, cities))

# Ergebnisse in das DataFrame einfügen
for index, country in enumerate(results):
    if country:
        country_df.at[country_df[country_df['country'].isnull()].index[index], 'country'] = country

# Schritt 3: Daten speichern
country_df.to_csv(r'C:\Users\Admin\Desktop\07 Python für Datenanalysten\Projekt_UFO\Nadja\countries.csv', index=False)


Fehler bei der Anfrage für Stadt 'lackland air force base tx': 401 Client Error: 401 for url: http://api.geonames.org/searchJSON?q=lackland%20air%20force%20base%20tx&maxRows=1&username=city_country
Fehler bei der Anfrage für Stadt 'gisborne (new zealand)': 401 Client Error: 401 for url: http://api.geonames.org/searchJSON?q=gisborne%20(new%20zealand)&maxRows=1&username=city_country
Fehler bei der Anfrage für Stadt 'mercedies': 401 Client Error: 401 for url: http://api.geonames.org/searchJSON?q=mercedies&maxRows=1&username=city_country
Fehler bei der Anfrage für Stadt 'holmes/pawling': 401 Client Error: 401 for url: http://api.geonames.org/searchJSON?q=holmes/pawling&maxRows=1&username=city_country
Fehler bei der Anfrage für Stadt 'gisborne (new zealand)': 401 Client Error: 401 for url: http://api.geonames.org/searchJSON?q=gisborne%20(new%20zealand)&maxRows=1&username=city_country
Fehler bei der Anfrage für Stadt 'lake macquarie (nsw&#44 australia)': 401 Client Error: 401 for url: http:/

KeyboardInterrupt: 